In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

tf.__version__

'2.9.0'

## MNT Model to convert English sentences to Vietnamese.

In [ ]:
## Getting Datasets
!curl -O -J https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.en
!curl -O -J https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.vi

!curl -O -J https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/vocab.en
!curl -O -J https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/vocab.vi

In [2]:
max_sentences = 100000

### Getting Engish Sentences

In [3]:
en_sentences = []

with open('./train.en', 'r', encoding='utf-8') as file:
    for i, each_line in enumerate(file):
        
        if i < 50:
            continue
            
        if i == max_sentences + 50:
            break
        
        sentence_split_by_spaces = each_line.strip()
        en_sentences.append(sentence_split_by_spaces)

### Getting Vietnamese Sentences

In [4]:
vi_sentences = []

with open('./train.vi', 'r', encoding='utf-8') as file:
    for i, each_line in enumerate(file):
        
        if i < 50:
            continue
            
        if i == max_sentences + 50:
            break
        
        sentence_split_by_spaces = each_line.strip()
        vi_sentences.append(sentence_split_by_spaces)

### Insertings tags on the start & end of each sentences

In [5]:
en_sentences = ['<s> ' + sentence.strip() + ' </s>' for sentence in en_sentences]

vi_sentences = ['<s> ' + sentence.strip() + ' </s>' for sentence in vi_sentences]

### Getting samples of sentences

In [6]:
for en, vi in zip(en_sentences[:2], vi_sentences[:2]):
    print(en)
    print("->")
    print(vi)
    print("\n\n")

<s> In each one of those assessments that we write , we always tag on a summary , and the summary is written for a non-scientific audience . </s>
->
<s> Trong mỗi bản đánh giá chúng tôi viết , chúng tôi luôn đính kèm một bản tóm lược , được viết cho những độc giả không chuyên về khoa học . </s>



<s> And we hand that summary to journalists and policy makers , in order to make headlines like these . </s>
->
<s> Chúng tôi đưa bản tóm lược cho các nhà báo và nhà chính sách để có được những dòng tít như thế này . </s>





### Splitting Dataset

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_en_sentences, valid_test_en_sentences, train_vi_sentences, valid_test_vi_sentences = train_test_split(en_sentences, 
                                                                                                            vi_sentences,
                                                                                                            test_size=0.2,
                                                                                                            shuffle=True)

In [9]:
valid_en_sentences, test_en_sentences, valid_vi_sentences, test_vi_sentences = train_test_split(valid_test_en_sentences,
                                                                                               valid_test_vi_sentences,
                                                                                               test_size=0.5,
                                                                                               shuffle=True)

In [10]:
print(f"Shape of Train set: {len(train_en_sentences)}")
print(f"Shape of Valid set: {len(valid_en_sentences)}")
print(f"Shape of Test set: {len(test_en_sentences)}")

Shape of Train set: 80000
Shape of Valid set: 10000
Shape of Test set: 10000


### Getting statistics on How long each sentences are

In [11]:
print("Lengths of English Sentences:")
pd.Series(train_en_sentences).str.split(" ").apply(len).describe(percentiles=[0.5, 0.75, 0.95])

Lengths of English Sentences:


count    80000.000000
mean        22.187625
std         14.734221
min          3.000000
50%         18.000000
75%         28.000000
95%         49.000000
max        630.000000
dtype: float64

In [12]:
n_en_seq = 50

In [13]:
print("Lengths of Vietnamese Sentences:")
pd.Series(train_vi_sentences).str.split(" ").apply(len).describe(percentiles=[0.5, 0.75, 0.95])

Lengths of Vietnamese Sentences:


count    80000.000000
mean        26.885437
std         18.675991
min          3.000000
50%         22.000000
75%         34.000000
95%         61.000000
max        852.000000
dtype: float64

In [14]:
n_vi_seq = 61

### Finding Unique number of Vocabs 

In [15]:
print("English Vocab:")

en_vocab = []

with open("./vocab.en", "r", encoding='utf-8') as file:
    for i, each_word in enumerate(file):
        
        # Removing of the unk token
        if i == 0:
            continue
        
        en_vocab.append(each_word.strip())

        
n_en_vocab = len(en_vocab)
print(f"Samples of English words: {en_vocab[:10]}")
print(f"Size of Engish vocab {n_en_vocab}")

English Vocab:
Samples of English words: ['<s>', '</s>', 'Rachel', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline']
Size of Engish vocab 17190


In [16]:
print("Vietnamese Vocab:")

vi_vocab = []

with open("./vocab.vi", "r", encoding='utf-8') as file:
    for i, each_word in enumerate(file):
        
        # Removing of the unk token
        if i == 0:
            continue
        
        vi_vocab.append(each_word.strip())

n_vi_vocab = len(vi_vocab)
print(f"Samples of Vietnamese words: {vi_vocab[:10]}")
print(f"Size of Vietnamese vocab {n_vi_vocab}")

Vietnamese Vocab:
Samples of Vietnamese words: ['<s>', '</s>', 'Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về']
Size of Vietnamese vocab 7708


### Training TextVectorizer Layer

In [17]:
encoder_text_vectorizer = layers.TextVectorization(max_tokens=n_en_vocab,
                                                  standardize=None,
                                                  split='whitespace',
                                                  output_sequence_length=n_en_seq,
                                                  name="encoder_text_vectorizer_layer")
## Needs to be a numPy array 
tmp = np.array(train_en_sentences)

encoder_text_vectorizer.adapt(tmp)

2023-01-08 22:15:00.645891: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-08 22:15:00.646133: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


2023-01-08 22:15:00.904516: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-08 22:15:00.932679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [18]:
example = "<s> This is a cat </s>"

print(f"Tokenized Form of \"{example}\":\n\n{encoder_text_vectorizer([example])}\n")
print(f"Samples from layer: {encoder_text_vectorizer.get_vocabulary()[:20]}")

Tokenized Form of "<s> This is a cat </s>":

[[   3   91   14   10 3706    4    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]

Samples from layer: ['', '[UNK]', ',', '<s>', '</s>', '.', 'the', 'to', 'of', 'and', 'a', 'that', 'I', 'in', 'is', 'you', 'it', '&apos;s', 'we', 'And']


In [22]:
decoder_text_vectorizer = layers.TextVectorization(max_tokens=n_vi_vocab,
                                                  standardize=None,
                                                  split='whitespace',
                                                  output_sequence_length=n_vi_seq - 1,
                                                  name="decoder_text_vectorizer_layer")
## Needs to be a numPy array
tmp = np.array(train_vi_sentences)

decoder_text_vectorizer.adapt(tmp)

2023-01-08 22:16:21.217617: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [23]:
example = "<s> Chúng tôi đưa bản tóm </s>"

print(f"Tokenized Form of \"{example}\":\n\n{decoder_text_vectorizer([example])}\n")
print(f"Samples from layer: {decoder_text_vectorizer.get_vocabulary()[:20]}")

Tokenized Form of "<s> Chúng tôi đưa bản tóm </s>":

[[   2   74    7  241  124 1525    3    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]

Samples from layer: ['', '[UNK]', '<s>', '</s>', ',', '.', 'là', 'tôi', 'một', 'có', 'và', 'những', 'chúng', 'của', 'ta', 'không', 'bạn', 'đó', 'người', 'trong']


### Creating a simple seq2seq model

In [24]:
## Encoder
encoder_input = layers.Input(shape=(1,) , dtype=tf.string, name="encoder_input")

x = encoder_text_vectorizer(encoder_input)
x = layers.Embedding(input_dim=n_en_vocab, output_dim=256, mask_zero=True, name="encoder_embedding")(x)
x = layers.GRU(256, return_sequences=True, name="encoder_gru_1")(x)

encoder_gru_last_layer, encoder_gru_last_state = layers.GRU(256, return_sequences=True, return_state=True, name="encoder_gru_last")(x)


encoder_model = tf.keras.models.Model(inputs=encoder_input, outputs=encoder_gru_last_layer, name="encoder_model")

## Decoder
decoder_input = layers.Input(shape=(1,), dtype=tf.string, name="decoder_input")

x = decoder_text_vectorizer(decoder_input)
x = layers.Embedding(input_dim=n_vi_vocab, output_dim=256, mask_zero=True, name="decoder_embedding")(x)
x = layers.GRU(256, return_sequences=True, name="decoder_gru_1")(x, initial_state=encoder_gru_last_state)
x = layers.GRU(256, return_sequences=True, name="decoder_gru_last")(x)

decoder_out = layers.Dense(n_vi_vocab, activation='softmax')(x)


seq2seq = tf.keras.models.Model(inputs=[encoder_model.inputs, decoder_input], outputs=decoder_out)
seq2seq.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [25]:
seq2seq.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 1)]          0           []                               
                                                                                                  
 encoder_text_vectorizer_layer   (None, 50)          0           ['encoder_input[0][0]']          
 (TextVectorization)                                                                              
                                                                                                  
 decoder_input (InputLayer)     [(None, 1)]          0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 50, 256)      4400640     ['encoder_text_vectorizer_l

### Preparing data for model

In [34]:
def prepare_data(X, y, tensor=False, batch_size=128):
    
    encoder_input =  np.array( X )
    decoder_input = np.array( [" ".join(sentence.split(" ")[:-1]) for sentence in y] )
    
    decoder_labels = [" ".join(sentence.split(" ")[1:]) for sentence in y]
    decoder_labels = decoder_text_vectorizer(decoder_labels).numpy()
    
    if tensor:
        encoder_input = tf.data.Dataset.from_tensor_slices(encoder_input)
        decoder_input = tf.data.Dataset.from_tensor_slices(decoder_input)
        decoder_labels = tf.data.Dataset.from_tensor_slices(decoder_labels)
        
        inputs = tf.data.Dataset.zip( (encoder_input, decoder_input) )
        labels = tf.data.Dataset.zip( (inputs, decoder_labels) ).batch(batch_size=batch_size).prefetch(tf.data.AUTOTUNE)
        
        return labels, None
    
    return (encoder_input, decoder_input), decoder_labels

In [ ]:
batch_size = 128

In [35]:
train_features, train_labels = prepare_data(train_en_sentences, train_vi_sentences, tensor=True)
valid_features, valid_labels = prepare_data(valid_en_sentences, valid_vi_sentences, tensor=True)
test_features, test_labels = prepare_data(test_en_sentences, test_vi_sentences, tensor=True)

### Training the model

In [36]:
epochs = 2

history = seq2seq.fit(train_features,
                      epochs=epochs, 
                      batch_size=batch_size,
                     validation_batch_size=batch_size,
                     validation_data=valid_features)

Epoch 1/2


2023-01-08 22:23:46.351360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-08 22:23:47.094698: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT8
    }
  }
}

	while inferring type of node 'cond_41/output/_22'
2023-01-08 22:23:47.097859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-08 22:23:47.407293: I tensorflow/cor

625/625 [==============================] - ETA: 0s - loss: 2.6159 - accuracy: 0.0507

2023-01-08 22:26:17.688570: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-08 22:26:18.121824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-08 22:26:18.271242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-08 22:26:18.416332: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-08 22:26:18.582482: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 163s 251ms/step - loss: 2.6159 - accuracy: 0.0507 - val_loss: 2.5642 - val_accuracy: 0.0534
Epoch 2/2
625/625 [==============================] - 152s 243ms/step - loss: 2.5331 - accuracy: 0.0748 - val_loss: 2.4331 - val_accuracy: 0.1078


### Inference Model

In [42]:
## Encoder
encoder_input = layers.Input(shape=(1,) , dtype=tf.string, name="encoder_input")

x = seq2seq.get_layer("encoder_text_vectorizer_layer")(encoder_input)
x = seq2seq.get_layer("encoder_embedding")(x)
x = seq2seq.get_layer("encoder_gru_1")(x)

encoder_gru_last_layer, encoder_gru_last_state = seq2seq.get_layer("encoder_gru_last")(x)


encoder_model = tf.keras.models.Model(inputs=encoder_input, outputs=[encoder_gru_last_layer, encoder_gru_last_state], name="encoder_model")





## Decoder
decoder_input = layers.Input(shape=(1,), dtype=tf.string, name="decoder_input")
decoder_encoder_state = layers.Input(shape=(256,))

x = seq2seq.get_layer("decoder_text_vectorizer_layer")(decoder_input)
x = seq2seq.get_layer("decoder_embedding")(x)

gru_1 = layers.GRU(256, return_sequences=True, name="decoder_gru_1")
x = gru_1(x, initial_state=decoder_encoder_state)

decoder_gru_last_layer = layers.GRU(256, return_sequences=True, name="decoder_gru_last")
gru_out = decoder_gru_last_layer(x)

decoder_out = seq2seq.get_layer("dense_1")(gru_out)


inference_model = tf.keras.models.Model(inputs=[decoder_input, decoder_encoder_state], outputs=[decoder_out, gru_out])
inference_model.compile()

gru_1.set_weights(seq2seq.get_layer("decoder_gru_1").get_weights())
decoder_gru_last_layer.set_weights(seq2seq.get_layer("decoder_gru_last").get_weights())

In [43]:
inference_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input (InputLayer)     [(None, 1)]          0           []                               
                                                                                                  
 decoder_text_vectorizer_layer   (None, 60)          0           ['decoder_input[0][0]']          
 (TextVectorization)                                                                              
                                                                                                  
 decoder_embedding (Embedding)  (None, 60, 256)      1973248     ['decoder_text_vectorizer_layer[5
                                                                 ][0]']                           
                                                                                            

In [77]:
word_ids_to_word = {key:value for key, value in enumerate(decoder_text_vectorizer.get_vocabulary())}

In [88]:
def generate_translation(english_text):
    
    #Generate content vector
    encoder_gru_last_layer, content_vector = encoder_model( np.array(english_text) )
    
    #y_pred. Starting value <s>
    y_pred = np.array([["<s>"]])
    predicted_sentence = []
    
    for _ in range(100):
        
        if y_pred[0][0] == '</s>':
            break
            
        softmax_probability, content_vector = inference_model.predict([y_pred, content_vector], verbose=0)
        # why
        content_vector = content_vector[:, 0, :]
        # Perform argmax on the last axis & get the highest value
        softmax_prediction = np.argmax(softmax_probability, axis=-1).ravel()[0]
        y_pred = np.array([[word_ids_to_word[softmax_prediction]]])
        
        predicted_sentence.append( word_ids_to_word[softmax_prediction] )
    
    return predicted_sentence

In [89]:
generate_translation(["Hello world"])

['Và']
['Và', 'Và']
['Và', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta'

['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', '

['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']
['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta']
['Và', 'Và', 'ta', 'Và', 'ta', '

['Và', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và', 'ta', 'Và']


['Và',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và',
 'ta',
 'Và']